### final summary
   >  - dowloaded the scripts from the CAS_PROD to test.cql
     - started the docker instances
     - inserted test.cql into docker C*
     - point the cas-pipeline to point to right EDMS and LCS env
     - launch CAS debug tool - http://cas-clue-renderer-debug.dev.cengage.info/build/launch.html#/debuginfo?resumeAttemptUrl=http://localhost:8181/services/cas/v2/attempts/2c315eb4-015f-1000-0003-d66665667e6e
    

### how to point docker CAS to an env
- flip EDMS 
    > `sed -i -e 's/qa-aetosapp/v2/g' /aetos/etc/com.cengage.clue.pipeline.cfg`
- flip LCS
    > `sed -i -e 's/lcs-v2-int/lcs-v2/g' /aetos/etc/com.cengage.lcs.client.rest.cfg`


In [2]:
import requests

def postponeDueDateForAttempt():
    url = "http://localhost:8181/services/cas/v2/attempts/2c315eb4-015f-1000-0003-d66665667e6e/duedate/1510300000000"
    payload = "{}"
    headers = {
        'content-type': "application/json",
        'cache-control': "no-cache",
        'postman-token': "474404f1-6f7e-861c-808d-6a0ad3c73143"
    }
    response = requests.request("POST", url, data=payload, headers=headers)
    return response



In [3]:
def process_payload(payload):
    try:
        jsonObject = json.loads(payload)
        global final
        completed =  jsonObject.get('completed', None)
        summary = jsonObject.get('summary')
        isSbs = None
        started = None
        startTime = jsonObject.get('startTime', None)
        if summary != None :
            isSbs = summary.get('stepByStepHelpEnabled')
            started = summary.get('started', None)
            
        attemptTreePath =  jsonObject.get('attemptTreePath', None)
        
        final = final.append([{'type':jsonObject[u'type'], 
                                'id':jsonObject[u'id'],
                                'scored':jsonObject[u'scored'],
                                'cgid':jsonObject[u'cgid'],
                                'completed':completed,
                                'isSbs' : isSbs,
                                'started' : started,
                                'startTime' : startTime,
                                'attemptTreePath' : attemptTreePath,
                                'duedate':jsonObject['initParams'][u'dueDate']
                              }], ignore_index=True)
    except ValueError:
        write_to_file(payload)
        write_to_file_newline()
        print("error")

In [30]:
# brew services start cassandra@2.2 
# load the cas data into cassandra
# !pip install cassandra-driver - to install driver
#  this will create tables
#  cluster.connect('cas_a_debug') - make sure you connect to right keyspace


from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
import pandas as pd
import json

pd.set_option('display.max_colwidth', -1)
final = pd.DataFrame(columns=['type','id','scored','cgid','completed','duedate'])

def write_to_file(stringObject):
    with open('/Users/mtony/Downloads/data.json', 'a') as f:
         json.dump(stringObject, f)
            
def write_to_file_newline():
    hs = open("/Users/mtony/Downloads/data.json","a")
    hs.write("\n")
    hs.close()

ap = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(protocol_version=3, auth_provider=ap)
# cluster = Cluster(['127.0.0.1'])  # provide contact points and port
session = cluster.connect('cas_a_debug')

session.execute('CREATE TABLE IF NOT EXISTS attempt (id text PRIMARY KEY, monotonic_timestamp bigint, payload text);')
session.execute('CREATE TABLE IF NOT EXISTS componentinstance (id text PRIMARY KEY, monotonic_timestamp bigint, payload text);')
rows = session.execute('select * from attempt limit 100;')
df = pd.DataFrame(list(rows))
df = df.sort_values('monotonic_timestamp')
# df
df.payload.apply(process_payload)
# final
# final[final.scored== False]
final[(final.type == 'clue:exerciseSet')]
# final[(final.type == 'clue:exerciseSetQuestion')]
# .attemptTreePath
# .describe()
# final[(final.type == 'clue:exerciseSetQuestion') & (final.scored == False)]
# final.describe()
# final.type.unique()


,attemptTreePath,cgid,completed,duedate,id,isSbs,scored,startTime,started,type
3,/563ef6dc-015f-1000-0003-5fdd57b3867c/563ef8a1-015f-1000-0003-5fdd57b3867c/563ef947-015f-1000-0000-5fdd57b3867c,LFNE80YEGQ2KVQDMN061,False,9223372036854775807,563ef947-015f-1000-0000-5fdd57b3867c,None,False,None,False,clue:exerciseSet
5,/563ef6dc-015f-1000-0003-5fdd57b3867c/563ef9b4-015f-1000-0003-5fdd57b3867c/563efa98-015f-1000-0003-5fdd57b3867c,CDFKGFX1E4BFD99FA327,False,9223372036854775807,563efa98-015f-1000-0003-5fdd57b3867c,None,False,None,False,clue:exerciseSet
17,/563ef6dc-015f-1000-0003-5fdd57b3867c/563efb08-015f-1000-0003-5fdd57b3867c/563efbdb-015f-1000-0002-5fdd57b3867c,FYXKD0A0CW7HWTDDB778,False,9223372036854775807,563efbdb-015f-1000-0002-5fdd57b3867c,None,False,None,False,clue:exerciseSet
31,/563ef6dc-015f-1000-0003-5fdd57b3867c/563ef70d-015f-1000-0003-5fdd57b3867c/563ef832-015f-1000-0003-5fdd57b3867c,FUZVQAPKX17JCAJZC490,False,9223372036854775807,563ef832-015f-1000-0003-5fdd57b3867c,None,False,2017-10-25T21:15:00.855-04:00,True,clue:exerciseSet


In [24]:
#
# use docker instances
# load the cas data into cassandra
# !pip install cassandra-driver - to install driver
#  this will create tables
#  cluster.connect('cas_a_debug' or 'cas_localhost') - make sure you connect to right keyspace


from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
import pandas as pd
import json

# pd.set_option('display.max_colwidth', -1)
final = pd.DataFrame(columns=['type','id','scored','cgid','completed','duedate'])

def write_to_file(stringObject):
    with open('/Users/mtony/Downloads/data.json', 'a') as f:
         json.dump(stringObject, f)
            
def write_to_file_newline():
    hs = open("/Users/mtony/Downloads/data.json","a")
    hs.write("\n")
    hs.close()
            

    

ap = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(protocol_version=3, auth_provider=ap)
# cluster = Cluster(['127.0.0.1'])  # provide contact points and port
session = cluster.connect('cas_localhost')

session.execute('CREATE TABLE IF NOT EXISTS attempt (id text PRIMARY KEY, monotonic_timestamp bigint, payload text);')
session.execute('CREATE TABLE IF NOT EXISTS componentinstance (id text PRIMARY KEY, monotonic_timestamp bigint, payload text);')
rows = session.execute('select * from attempt limit 100;')
df = pd.DataFrame(list(rows))
df = df.sort_values('monotonic_timestamp')
# df
df.payload.apply(process_payload)
xx = final
xx['startTime'] =  pd.to_datetime(xx['startTime'], format='%Y-%m-%dT%H:%M:%S.%f')
# 2017-10-25T21:15:00.855-04:00
xx = xx.sort_values('startTime')
# xx
xx.to_csv('/Users/mtony/Downloads/data-output.csv')

#------------ duplicated - value
# final[final.cgid.duplicated()]

# final[(final.cgid == 'PWLAERAZA9YJQXTQR113')]
# final[(final.cgid == 'WPPRRY5Y3J280A910886')]


# final[(final.cgid == 'HZJNPG3B831KYX9YB490') | (final.cgid == 'NWQNPRA3UAHHJJPFQ070')]
# final[(final.type == 'clue:exerciseSet')]
# final[(final.type == 'clue:exerciseSetQuestion')]


# .attemptTreePath
# final[(final.type == 'clue:exerciseSetQuestion') & (final.scored == False)]
# final.describe()
# final.type.unique()